In [1]:
#importing libraries
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
import os
import PIL
import random
from time import time
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy

In [2]:
#Training and Testing datasets paths
train_path = "../input/sprints-ai-and-ml-competition-2022/Scenes training set/Scenes training set"
test_path = "../input/sprints-ai-and-ml-competition-2022/Scenes testing test/Scenes testing test"

In [3]:
img_width = 150 #Determine width
img_height = 150  #Determine height
batch_size = 32  #Determine Batch size

In [4]:
# Loading data images and labels function for training
def load_img(pth):
    Img = []
    Lbl = []  
    lbl = 0       
    for i in os.listdir(pth): 
          print(i)
          if i == 'buildings': 
                lbl = 0
          elif i == 'forest':
                lbl = 1
          elif i == 'glacier':
                lbl = 2
          elif i == 'mountain':
                lbl = 3
          elif i == 'sea':
                lbl = 4
          elif i == 'street':
                lbl = 5
            
          for m in os.listdir(pth+'/'+i): 
               
                image = cv2.imread(pth+'/'+i+'/'+m) 
                image = cv2.resize(image,(150,150)) 
                Img.append(image)
                Lbl.append(lbl)
        
    return shuffle(Img, Lbl, random_state = 1) 

def class_lbl(index):
    labels = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea',5:'street' }
    
    return labels[index]

In [5]:
# Converting images and labels into NumPy arrays
img, lbl = load_img(train_path) 
img = np.array(img) 
lbl = np.array(lbl)

In [6]:
#Plotting random images with thier labels
fig ,ax = plt.subplots(3,3)
fig.subplots_adjust(0,0,3,3)
for n in range (3):
    for i in range(3):
        var = random.randint(0,len(img))
        ax[n,i].imshow(img[var])
        ax[n,i].set_title(class_lbl(lbl[var]))
    

In [7]:
#Building the model
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(150,150,3),
                   pooling='avg',classes=6,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

In [8]:
# Adding some layer (Tuning the model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dropout(0.2))
resnet_model.add(Dense(6, activation='softmax'))

In [9]:
#Model Summary
resnet_model.summary()

In [10]:
#Compiling the model
resnet_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [11]:
#Fitting the model
resnet_model.fit(img,lbl,epochs = 5,validation_split=0.2)

In [12]:
#Reading the empty submit file
df = pd.read_csv('../input/sprints-ai-and-ml-competition-2022/submit.csv')

In [14]:
# Predicting the test set and storing the predictions in a DataFrame
from tensorflow import expand_dims
images=os.listdir(test_path)

for i,image in enumerate(images):
  arr=cv2.imread(os.path.join(test_path,image))
  arr = cv2.resize(arr, (150, 150))
  arr = expand_dims(arr, axis=0)
  name=image
  shape=arr.shape
  prediction=np.argmax(resnet_model.predict(arr))
  df.loc[i,"Image"]=str(name)
  df.loc[i,"Label"]=prediction

In [17]:
# Appending results to a CSV file
df.to_csv('./submit50.csv',index = False)